### This script generates questions to evaluate conflicting news
##### The questions bring out the differences in the conflicting news
##### We use ChatGPT to identify these questions and evaluate the quality of the summary by answering these questions.
##### The quality of the summary is evaluated based on the cosine similarity difference between answers achieved by just reading the summary vs. reading individual news articles

In [1]:
import pandas as pd
import statistics

from bert_score import BERTScorer
from rouge_score import rouge_scorer
import ast
from nltk.tokenize import sent_tokenize

import sys

/home/ahaque2/venv/py3_scads_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from openai import OpenAI

import os
from openai import OpenAI

def get_chatpgt_scores(prompt):

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="gpt-4o",
    )
    return chat_completion.choices[0].message.content

os.environ["LAS_API_TOKEN"] = "a4adfef6de531c1c258c9eda6f9be5c7a5701e365c32babe72273f63214811df"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("LAS_API_TOKEN"),
)

In [3]:
df_news = pd.read_csv('../data/conflicting_news_dataset.csv')
document = 'News'

In [4]:
conflict_q_prompts = open('../prompts/identify_conflict_questions.txt').read()

In [6]:
questions = []

## We iterate over all news clusters (each cluster contains a news from a Left and a Right perspective). 
## Change the range if you wish to run it for a different set of news clusters 
for i in range(1,16):

    news = df_news[df_news.cluster_id == i]
    docs = [news[news.side == 'L'].text.values[0], news[news.side == 'R'].text.values[0]]
    
    conflict_q_prompts = conflict_q_prompts + "\n" + document + " source 1: " + docs[0] + "\n\n=======\n\n" + document + " source 2: " + docs[1]
    Q = get_chatpgt_scores(conflict_q_prompts)
    
    questions.append(Q)

In [ ]:
df_questions = pd.DataFrame()
df_questions['cluster'] = list(range(1,16))
df_questions['questions'] = questions

In [ ]:
df_questions.to_csv('../data/news_questions.csv')